# Notebook 2 - Crear y consultar una Base de Datos Relacional con MySQL

## Ejercicio 1

<b>Requisito:</b>
- Descargar el archivo <a href="./sakila-db.zip">sakila-db.zip</a>

<b>Problema: Un poco de retro-ingeniería...</b>

Tienen a su disposición el archivo sakila-db.zip, lo cual contiene dos archivos:
- 'sakila-schema.sql' : una serie de instrucciones en lenguaje SQL para crear una Base de Datos llamada 'sakila'
- 'sakila-data.sql' : una serie de instrucciones en SQL para insertar datos en la Base de Datos

A partir de estos archivos, el ejercicio consiste en deducir el Modelo Lógico de Datos (Modelo Relacional) y el Modelo Conceptual de Datos (Modelo Entidad-Asociación).

- MLD: <b>por completar</b>

- MCD: <b>por completar</b>

- ¿De qué podría servir esta base de datos? ¿Cuál era su aplicación probable? ¿Cuál sería una adaptación posible más moderna? 
<b>por responder</b>

## Ejercicio 2

<b>Requisito:</b>
- Haber terminado el ejercicio 2

<b>Problema: ¿Cómo importar una base de datos?</b>

Queremos cargar la base de datos 'sakila' a partir de los 2 archivos .sql a nuestra disposición. ¿Cómo podemos hacer?

<b>Ayuda:</b> Revisar la página de documentación de MySQL que explica cómo ejecutar instrucciones SQL desde un archivo https://dev.mysql.com/doc/refman/8.0/en/mysql-batch-commands.html

## Ejercicio 3

<b>Requisito:</b>
- Haber terminado el ejercicio 3 y haber importado los datos en su MySQL local.

<b>Problema: Realizar consultas <code>SELECT</code> simple</b>

Desde un entorno Python (o MySQL CLI o MySQL Workbench), realizar los comandos SQL que permiten responder a las consultas siguientes. (No olviden utilizar el comando SQL <code>USE [database-name]</code> ;-))

<b> Ayuda: http://www.mysqltutorial.org/basic-mysql-tutorial.aspx</b>

1) A partir de la tabla "film", mostrar solamente cuáles son los títulos de pelicula disponibles.

2) A partir de la tabla "film", mostrar de qué años son las peliculas disponibles. NB: utilizar la instrucción <code>DISTINCT</code>

3) Mostrar los títulos de pelicula en la tabla "film" que se puede arrendar por más de 4 días (rental_duration)

4) En la tabla "payment", mostrar el customer_id, rental_id, amount y payment_date de todos los arriendos realizados entre las fechas 2005-07-01 y 2005-08-01

In [4]:
import mysql.connector
import pandas as pd
from IPython.display import display,HTML

#reemplazar 'root' por el password del usuario administrador de MySQL
db_connection = mysql.connector.connect(user="root",host="localhost",password="root")
cursor = db_connection.cursor()
cursor.execute("USE sakila")

In [5]:
df=pd.read_sql('SELECT title,release_year FROM film LIMIT 5;',con=db_connection)
display(df)

,title,release_year
0,ACADEMY DINOSAUR,2006
1,ACE GOLDFINGER,2006
2,ADAPTATION HOLES,2006
3,AFFAIR PREJUDICE,2006
4,AFRICAN EGG,2006


## Ejercicio 4

<b>Requisito:</b>
- Haber terminado el ejercicio 4

<b>Problema: Realizar consultas <code>SELECT</code> con funciones de agregación </b>

Desde un entorno Python (o MySQL CLI o MySQL Workbench), realizar los comandos SQL que permiten responder a las consultas siguientes.

<b> Ayuda: http://www.mysqltutorial.org/basic-mysql-tutorial.aspx</b>


1) Contar el número de clientes en la tabla "CUSTOMER"

2) En la tabla "RENTAL", contar cuánto arriendos se realizó por cada cliente (NB: utilizar la instucción <code>GROUP BY</code> y <code>COUNT</code>)

3) ¿Qué devuelve la consulta SQL siguiente?

<code>SELECT MAX(total) FROM (SELECT customer_id,count(*) as total FROM rental GROUP BY customer_id) as t1;</code>

4) ¿Qué es un sub-consulta?

5) En la tabla "payment", mostrar el volumen de negocio realizado por cada miembro del staff (staff_id) en total.

6) En una consulta, mostrar el staff_id que ha realizado más volumen de negocio el dia 2005-08-23.

In [21]:
df=pd.read_sql('SELECT COUNT(*) FROM customer;',con=db_connection)
display(df)

,COUNT(*)
0,599


In [3]:
df=pd.read_sql('SELECT customer_id as idCliente,count(*) as total FROM rental GROUP BY customer_id LIMIT 5;',con=db_connection)
display(df)

,idCliente,total
0,1,32
1,2,27
2,3,26
3,4,22
4,5,38


In [37]:
df=pd.read_sql('SELECT MAX(total) FROM (SELECT customer_id,count(*) as total FROM rental GROUP BY customer_id) as t1;',con=db_connection)
display(df)

,MAX(total)
0,46


<b>3)</b> Lo que hace la consulta anterior es realizar una subconsulta para poder encontrar el máximo de arriendos que ha podido hacer alguno cliente

</b>4)</b> Una subconsulta es cuando se hace una instrucción de consulta (SELECT, INSERT, etc.) contenida dentro de otra consulta

In [45]:
df=pd.read_sql('SELECT staff_id as idStaff,count(*) as total FROM payment GROUP BY staff_id;',con=db_connection)
display(df)

,idStaff,total
0,1,8057
1,2,7992


In [69]:
df=pd.read_sql('SELECT staff_id,MAX(total) FROM (SELECT staff_id,count(*) as total FROM payment \
WHERE payment_date between "2005-08-23" and "2005-08-24" GROUP BY staff_id) as t2;',con=db_connection)
display(df)

,staff_id,MAX(total)
0,1,331


## Ejercicio 5

<b>Requisito:</b>
- Haber terminado el ejercicio 4

<b>Problema: Realizar consultas <code>SELECT</code> con operaciones de JOIN </b>

Desde un entorno Python (o MySQL CLI o MySQL Workbench), realizar los comandos SQL que permiten responder a las consultas siguientes.

<b> Ayuda: http://www.mysqltutorial.org/basic-mysql-tutorial.aspx</b>


1) En las peliculas "Film" y "Language", mostrar el título de la pelicula y el nombre de su idioma. (NB: utilizar una operación de INNER JOIN)

2) Mostrar los nombres y apellidos de los clientes que han arrendado la pelicula "CHICAGO NORTH"

3) Mostrar por cada nombres y apellidos de cliente, en qué ciudad viven (tabla Customer y City)

4) Mostrar cuántas veces fueron arrendadas las peliculas (mostrando el título de la pelicula)

5) Mostrar qué pelicula generó más volumen de negocio

6) Mostrar los clientes que arrendar

In [7]:
df=pd.read_sql('SELECT title,name FROM film INNER JOIN language \
on film.language_id=language.language_id LIMIT 5;',con=db_connection)
display(df)

,title,name
0,ACADEMY DINOSAUR,English
1,ACE GOLDFINGER,English
2,ADAPTATION HOLES,English
3,AFFAIR PREJUDICE,English
4,AFRICAN EGG,English


In [4]:
df=pd.read_sql('SELECT first_name,last_name FROM customer INNER JOIN rental ON \
customer.customer_id=rental.customer_id INNER JOIN inventory ON rental.inventory_id=inventory.inventory_id \
INNER JOIN film ON inventory.film_id=film.film_id WHERE film.title=\"CHICAGO NORTH\" LIMIT 5;',con=db_connection)
display(df)

,first_name,last_name
0,ARNOLD,HAVENS
1,PENNY,NEAL
2,MARGIE,WADE
3,ALVIN,DELOACH
4,CAROLE,BARNETT


In [103]:
df=pd.read_sql('SELECT first_name,last_name,city FROM customer INNER JOIN address \
ON customer.address_id=address.address_id INNER JOIN city ON address.city_id=city.city_id LIMIT 5;',con=db_connection)
display(df)

,first_name,last_name,city
0,MARY,SMITH,Sasebo
1,PATRICIA,JOHNSON,San Bernardino
2,LINDA,WILLIAMS,Athenai
3,BARBARA,JONES,Myingyan
4,ELIZABETH,BROWN,Nantou


In [8]:
df=pd.read_sql('SELECT title,count(*) FROM film INNER JOIN inventory \
ON film.film_id=inventory.film_id INNER JOIN rental ON inventory.inventory_id=rental.inventory_id \
GROUP BY title LIMIT 5;',con=db_connection)
display(df)

,title,count(*)
0,ACADEMY DINOSAUR,23
1,ACE GOLDFINGER,7
2,ADAPTATION HOLES,12
3,AFFAIR PREJUDICE,23
4,AFRICAN EGG,12


In [6]:
df=pd.read_sql('SELECT title,MAX(total) FROM (SELECT title,count(*) as total FROM film INNER JOIN inventory \
ON film.film_id=inventory.film_id INNER JOIN rental ON inventory.inventory_id=rental.inventory_id \
GROUP BY title) as t3;',con=db_connection)
display(df)

,title,MAX(total)
0,ACADEMY DINOSAUR,34


In [7]:
df = pd.read_sql('SELECT DISTINCT first_name,last_name FROM customer \
INNER JOIN rental ON customer.customer_id=rental.customer_id \
LIMIT 10;',con=db_connection)
display(df)

,first_name,last_name
0,MARY,SMITH
1,PATRICIA,JOHNSON
2,LINDA,WILLIAMS
3,BARBARA,JONES
4,ELIZABETH,BROWN
5,JENNIFER,DAVIS
6,MARIA,MILLER
7,SUSAN,WILSON
8,MARGARET,MOORE
9,DOROTHY,TAYLOR
